In [ ]:
import numpy as np
from PIL import Image
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from ast import literal_eval

def GetM3(L,C,R,T):
    # marginalization into borders
    L = np.clip(L, -T, T).flatten('F')
    C = np.clip(C, -T, T).flatten('F')
    R = np.clip(R, -T, T).flatten('F')

    # get cooccurences [-T...T]
    M = np.zeros((2*T+1,2*T+1,2*T+1))
    for i in range(-T, T+1, 1):
        C2 = C[L==i]
        R2 = R[L==i]
        for j in range(-T, T+1, 1):
            R3 = R2[C2==j]
            for k in range(-T, T+1, 1):
                M[i+T,j+T,k+T] = np.sum(R3==k)

    # normalization
    M = M.flatten('F')
    M /= np.sum(M)
    return M

def spam_extract_2(X, T):
    # horizontal left-right
    X = np.concatenate((X[:,:,0], X[:,:,1], X[:,:,2]), axis=1)
    D = X[:,:-1] - X[:,1:]
    L = D[:,2:]
    C = D[:,1:-1]
    R = D[:,:-2]
    Mh1 = GetM3(L.copy(),C.copy(),R.copy(),T)

    # horizontal right-left
    D = -D;
    L = D[:,:-2]
    C = D[:,1:-1]
    R = D[:,2:]
    Mh2 = GetM3(L.copy(),C.copy(),R.copy(),T)

    # vertical bottom top
    D = X[:-1,:] - X[1:,:]
    L = D[2:,:]
    C = D[1:-1,:]
    R = D[:-2,:]
    Mv1 = GetM3(L.copy(),C.copy(),R.copy(),T)

    # vertical top bottom
    D = -D
    L = D[:-2,:]
    C = D[1:-1,:]
    R = D[2:,:]
    Mv2 = GetM3(L.copy(),C.copy(),R.copy(),T)

    # diagonal left-right
    D = X[:-1,:-1] - X[1:,1:]
    L = D[2:,2:]
    C = D[1:-1,1:-1]
    R = D[:-2,:-2]
    Md1 = GetM3(L.copy(),C.copy(),R.copy(),T)

    # diagonal right-left
    D = -D
    L = D[:-2,:-2]
    C = D[1:-1,1:-1]
    R = D[2:,2:]
    Md2 = GetM3(L.copy(),C.copy(),R.copy(),T)

    # minor diagonal left-right
    D = X[1:,:-1] - X[:-1,1:]
    L = D[:-2,2:]
    C = D[1:-1,1:-1]
    R = D[2:,:-2]
    Mm1 = GetM3(L.copy(),C.copy(),R.copy(),T)

    # minor diagonal right-left
    D = -D
    L = D[2:,:-2]
    C = D[1:-1,1:-1]
    R = D[:-2,2:]
    Mm2 = GetM3(L.copy(),C.copy(),R.copy(),T)

    F1 = (Mh1+Mh2+Mv1+Mv2)/4
    F2 = (Md1+Md2+Mm1+Mm2)/4
    F = np.concatenate((F1, F2), axis=0)
    return F

def loader(path):
    with open(path, 'rb') as f:
        with Image.open(f) as img:
            return img.convert('RGB')

def SPAM(path):
     return spam_extract_2(np.array(loader(path), dtype='float'),3)

In [ ]:
import os
import pandas as pd

# Specify the paths to your folders
cover_folder = './data/cover/'
stego_folder = './data/stego/'

# Create empty lists to store image paths and labels
image_paths = []
labels = []

# Function to get file paths and labels from a folder
def get_files_and_labels(folder, label):
    files = os.listdir(folder)
    file_paths = [os.path.join(folder, file) for file in files]
    return file_paths, [label] * len(file_paths)

# Get image paths and labels for 'cover' folder
cover_paths, cover_labels = get_files_and_labels(cover_folder, 'cover')
image_paths.extend(cover_paths)
labels.extend(cover_labels)

# Get image paths and labels for 'stego' folder
stego_paths, stego_labels = get_files_and_labels(stego_folder, 'stego')
image_paths.extend(stego_paths)
labels.extend(stego_labels)

# Create a DataFrame
df = pd.DataFrame({'Image_Path': image_paths, 'Label': labels})

In [ ]:
df.head()

In [ ]:
df_randomized = df.sample(frac=1).reset_index(drop=True)

In [ ]:
df_randomized.head()

In [ ]:
import pandas as pd
from tqdm import tqdm

tqdm.pandas()
df['Spam_Features'] = df['Image_Path'].progress_apply(SPAM)

In [ ]:
X = new_df['Spam_Features'].tolist()
y = new_df['Label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

kernels = ['linear', 'poly', 'rbf', 'sigmoid']

for kernel in kernels:
    svm_classifier = SVC(kernel=kernel)
    svm_classifier.fit(X_train_scaled, y_train)
    y_pred = svm_classifier.predict(X_test_scaled)

    accuracy = accuracy_score(y_test, y_pred)
    print(f"Accuracy with {kernel} kernel: {accuracy:.2%}")